# Load Packages

In [1]:
import numpy as np
from scipy.linalg import eigh, block_diag, pinv
from scipy.linalg import eig
import torch
import pymanopt
from pymanopt.manifolds import Product, Stiefel
from pymanopt import Problem
from pymanopt.optimizers import ConjugateGradient
import autograd
from scipy.linalg import qr
from numpy.linalg import qr
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist, squareform
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import eigh, block_diag, pinv
import torch
import pymanopt
from pymanopt.manifolds import Product, Stiefel
from pymanopt import Problem
from pymanopt.optimizers import ConjugateGradient
import autograd
from scipy.stats import ortho_group
import matplotlib.pyplot as plt
import seaborn as sns
import os
from joblib import Parallel, delayed

In [2]:
import zipfile

# Functions

## Methods

In [3]:
def method1(X, Y1, Y2, d1, a, l):
    """
    Compute the desired matrices and eigenvectors as described in the prompt.

    Parameters:
        X (np.ndarray): Data matrix (n x p)
        Y1 (np.ndarray): Binary response vector of length n
        Y2 (np.ndarray): Binary response vector of length n
        d1 (int): Number of eigenvalues/vectors for the first block
        d2 (int): Number of eigenvalues/vectors for the second block
        a (float): hyperparameter for contrastive term
        l (float): hyperparameter for PCA term
    Returns:
        tuple: V1, V2 matrices corresponding to the extracted eigenvectors
    """
    # Total number of eigenvalues/vectors to extract
    d = d1
    # Check dimensions
    n, p = X.shape
    assert len(Y1) == n and len(Y2) == n, "Response vectors must have length n"
    assert d < p, "d must be less than the number of features p"
    assert a >= 0, "contrastive hyperparameter must be nonnegative"
    assert l >= 0, "PCA hyperparameter must be nonnegative"
    
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    
    epsilon = 1e-4  # Small positive value
    Sigma_inv = np.linalg.inv(Sigma + epsilon * np.eye(Sigma.shape[0]))
    
    print(1)

    def compute_sigma_1(X, Y):
        """Compute Sigma_1 (or Sigma_2) as described."""
        p_1 = np.mean(Y)  # Proportion of slice Y = 1
        m_1 = np.mean(X[Y == 1], axis=0)  # Mean for Y = 1
        m_0 = np.mean(X[Y == 0], axis=0)  # Mean for Y = 0
        
        # Compute Sigma_1
        Sigma_1 = p_1 * np.outer(m_1, m_1) + (1 - p_1) * np.outer(m_0, m_0)
        return Sigma_1

    # Compute Sigma_1, Sigma_2, and Sigma_12
    Sigma_1 = compute_sigma_1(X, Y1)
    Sigma_2 = compute_sigma_1(X, Y2)
    
    print(2)
    
    # Compute A_1, A_2, A_12
    A_1 = Sigma @ Sigma_1 @ Sigma
    A_2 = Sigma @ Sigma_2 @ Sigma
    
    print(3)
    
    # Compute matrix to find top eigenvectors of
    M1 = Sigma_inv @ (a * A_1 - A_2) @ Sigma_inv + l * Sigma_inv
    M2 = Sigma_inv @ Sigma_inv
    
    M1 = (M1 + M1.T)/2
    M2 = (M2 + M2.T)/2
    
    print(4)
    
    #print("Symmetry check for M1:", np.allclose(M1, M1.T))
    #print("Symmetry check for M2:", np.allclose(M2, M2.T))

    
    #M2_cond_number = np.linalg.cond(M2)
    #print("M2 condition number = ", end="")
    #print(M2_cond_number)
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eigh(M1, M2)
    
    print(5)
    
    # Coerce eigenvalues and eigenvectors to real values
    eigenvalues = np.real(eigenvalues)
    eigenvectors = np.real(eigenvectors)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    
    #print("all eigenvalues = ", end="")
    #print(eigenvalues)
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvalues = eigenvalues[:d1]
    top_eigenvectors = eigenvectors[:, :d1]
    
    print(6)
    
    #print("top eigenvalues = ", end="")
    #print(top_eigenvalues)
    
    #print("pct var = ", end="")
    #print(sum(top_eigenvalues) / sum(eigenvalues) * 100)
    
    # Rotate back
    V = Sigma_inv @ top_eigenvectors
    
    print(7)
    
    # Perform QR decomposition to orthogonalize V
    Q, R = np.linalg.qr(V)
    
    return np.real(Q)

In [4]:
def method2(X, Y1, Y2, d1, d2, e, a, b, c, l):
    """
    Compute the desired matrices and eigenvectors as described in the prompt.

    Parameters:
        X (np.ndarray): Data matrix (n x p)
        Y1 (np.ndarray): Binary response vector of length n
        Y2 (np.ndarray): Binary response vector of length n
        d1 (int): Number of eigenvalues/vectors for the first block
        d2 (int): Number of eigenvalues/vectors for the second block
        a (float): hyperparameter for contrastive term (alpha)
        b (float): hyperparameter for V2 (beta)
        c (float): hyperparameter for V2 contrastive term (gamma)
        l (float): hyperparameter for PCA term (lambda)
    Returns:
        tuple: V1, V2 matrices corresponding to the extracted eigenvectors
    """
    # Total number of eigenvalues/vectors to extract
    d = d1 + d2
    # Check dimensions
    n, p = X.shape
    assert len(Y1) == n and len(Y2) == n, "Response vectors must have length n"
    assert d < p, "d must be less than the number of features p"
    assert a >= 0, "contrastive hyperparameter must be nonnegative"
    assert b >= 0, "hyperparameter for V2 must be nonnegative"
    assert c >= 0, "hyperparameter for V2 contrastive term must be nonnegative"
    assert l >= 0, "PCA hyperparameter must be nonnegative"
    assert e >= 0, "first hyperparameter must be nonnegative"
    assert max(a, b, c, l, e) >= 0, "at least one hyperparameter must be positive"
    
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    #Sigma_inv = np.linalg.inv(Sigma)
    B = Sigma @ Sigma

    def compute_sigma_1(X, Y):
        """Compute Sigma_1 (or Sigma_2) as described."""
        p_1 = np.mean(Y)  # Proportion of slice Y = 1
        m_1 = np.mean(X[Y == 1], axis=0)  # Mean for Y = 1
        m_0 = np.mean(X[Y == 0], axis=0)  # Mean for Y = 0
        
        # Compute Sigma_1
        Sigma_1 = p_1 * np.outer(m_1, m_1) + (1 - p_1) * np.outer(m_0, m_0)
        return Sigma_1

    # Compute Sigma_1, Sigma_2, and Sigma_12
    Sigma_1 = compute_sigma_1(X, Y1)
    Sigma_2 = compute_sigma_1(X, Y2)
    
    # Compute A_1, A_2, A_12
    A_1 = Sigma @ Sigma_1 @ Sigma
    A_2 = Sigma @ Sigma_2 @ Sigma
    
    # V1
    # Compute matrix to find top eigenvectors of
    M1 = Sigma @ (A_1 - a * A_2) @ Sigma
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors1 = np.linalg.eig(M1)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors1 = eigenvectors1[:, sorted_indices]
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvectors1 = eigenvectors1[:, :d1]
    
    # Rotate back
    V1 = np.linalg.solve(Sigma, top_eigenvectors1)
    
    # Perform QR decomposition to orthogonalize V
    Q1, R = np.linalg.qr(V1)
    
    
    # V2
    # Compute matrix to find top eigenvectors of
    M2 = Sigma @ (c * A_2 - b * A_1) @ Sigma
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors2 = np.linalg.eig(M2)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors2 = eigenvectors2[:, sorted_indices]
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvectors2 = eigenvectors2[:, :d1]
    
    # Rotate back
    V2 = np.linalg.solve(Sigma, top_eigenvectors2)
    
    # Perform QR decomposition to orthogonalize V
    Q2, R = np.linalg.qr(V2)
    
    if l == 0:
        return np.real(Q1), np.real(Q2)
    
    else:
        
        # Convert necessary matrices to PyTorch tensors
        Sigma_cubed = torch.tensor(Sigma @ B, dtype=torch.float64)
        A_1_torch = torch.tensor(A_1, dtype=torch.float64)
        A_2_torch = torch.tensor(A_2, dtype=torch.float64)
        B_torch = torch.tensor(B, dtype=torch.float64)
        
        # Define the product manifold
        manifold = pymanopt.manifolds.Product([
            pymanopt.manifolds.Stiefel(p, d1),
            pymanopt.manifolds.Stiefel(p, d2)
        ])

        
        @pymanopt.function.pytorch(manifold)
        def cost(V_1, V_2):
            V_combined = torch.cat([V_1, V_2], dim=1)
            part1 = -torch.trace(V_1.T @ A_1_torch @ V_1)
            part2 = -torch.trace(V_2.T @ A_2_torch @ V_2)
            VDV = V_combined.T @ B_torch @ V_combined
            VDV_inv = torch.linalg.pinv(VDV)
            part3 = -l * torch.trace(V_combined.T @ Sigma_cubed @ V_combined @ VDV_inv)
            return part1 + part2 + part3

        # Create the optimization problem
        problem = Problem(manifold=manifold, cost=cost)

        # Use a conjugate gradient optimizer
        optimizer = ConjugateGradient()
        result = optimizer.run(problem)

        # Extract optimized V_1 and V_2
        V_1_opt, V_2_opt = result.point
        
        # Rotate back
        V1 = np.linalg.solve(Sigma, V_1_opt)
        V2 = np.linalg.solve(Sigma, V_2_opt)
        
        # Perform QR decomposition to orthogonalize V
        Q1, R = np.linalg.qr(V1)
        Q2, R = np.linalg.qr(V2)
        
        return Q1, Q2

In [5]:
def fairPCA(X, Y, d, m):
    # Compute the covariance matrix
    Sigma = np.cov(X, rowvar=False)
    
    # Split the data into two groups based on Y
    X0 = X[Y == 0]
    X1 = X[Y == 1]
    
    # Compute mean vectors
    mu0 = np.mean(X0, axis=0)
    mu1 = np.mean(X1, axis=0)
    
    # Compute covariance matrices
    Sigma0 = np.cov(X0, rowvar=False)
    Sigma1 = np.cov(X1, rowvar=False)
    
    # Compute the difference of covariance matrices
    Sigma_diff = Sigma0 - Sigma1
    
    # Compute eigenvalues and eigenvectors of Sigma_diff
    eigvals, eigvecs = np.linalg.eigh(Sigma_diff)
    
    # Sort eigenvectors by absolute eigenvalues in descending order
    sorted_indices = np.argsort(-np.abs(eigvals))
    top_m_eigvecs = eigvecs[:, sorted_indices[:m]]
    
    # Include mu0 - mu1 in the subspace
    mu_diff = (mu0 - mu1).reshape(-1, 1)
    combined_basis = np.hstack([top_m_eigvecs, mu_diff])
    
    # Perform QR decomposition to obtain an orthonormal basis
    Q, _ = np.linalg.qr(combined_basis, mode='reduced')
    Pi = Q @ Q.T  # Projection matrix
    
    # Compute Pi Sigma Pi
    transformed_Sigma = Pi @ Sigma @ Pi
    
    # Compute the top d eigenvectors of Pi Sigma Pi
    eigvals_final, eigvecs_final = np.linalg.eigh(transformed_Sigma)
    sorted_indices_final = np.argsort(-eigvals_final)
    top_d_eigvecs = eigvecs_final[:, sorted_indices_final[:d]]
    
    return top_d_eigvecs

In [6]:
def SIR(X, Y, d):
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    
    epsilon = 1e-5  # Small positive value
    Sigma_inv = np.linalg.inv(Sigma + epsilon * np.eye(Sigma.shape[0]))

    def compute_sigma_1(X, Y):
        """Compute Sigma_1 (or Sigma_2) as described."""
        p_1 = np.mean(Y)  # Proportion of slice Y = 1
        m_1 = np.mean(X[Y == 1], axis=0)  # Mean for Y = 1
        m_0 = np.mean(X[Y == 0], axis=0)  # Mean for Y = 0
        
        # Compute Sigma_1
        Sigma_1 = p_1 * np.outer(m_1, m_1) + (1 - p_1) * np.outer(m_0, m_0)
        return Sigma_1

    # Compute Sigma_1, Sigma_2, and Sigma_12
    Sigma_1 = compute_sigma_1(X, Y)
    
    # Compute A_1, A_2, A_12
    A_1 = Sigma @ Sigma_1 @ Sigma
    
    # Compute matrix to find top eigenvectors of
    M = Sigma_inv @ A_1 @ Sigma_inv
    
    M = (M + M.T)/2

    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eig(M)
    
    # Coerce eigenvalues and eigenvectors to real values
    eigenvalues = np.real(eigenvalues)
    eigenvectors = np.real(eigenvectors)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvalues = eigenvalues[:d]
    top_eigenvectors = eigenvectors[:, :d]
    
    # Rotate back
    V = Sigma_inv @ top_eigenvectors
    
    # Perform QR decomposition to orthogonalize V
    Q, R = np.linalg.qr(V)
    
    return np.real(Q)

## Evaluation

In [7]:
def mmd(X1, X2, sigma, batch_size=1000):
    """
    Compute the Maximum Mean Discrepancy (MMD) between two datasets X1 and X2 with memory efficiency.
    """
    def rbf_batch(X1, X2, sigma):
        """
        Compute the Radial Basis Function (RBF) kernel between two datasets in batches.
        """
        result = 0.0
        for i in range(0, X1.shape[0], batch_size):
            X1_batch = X1[i:i + batch_size]
            distances = cdist(X1_batch, X2, 'sqeuclidean')
            result += np.sum(np.exp(-distances / (2 * sigma**2)))
        return result

    m = X1.shape[0]
    n = X2.shape[0]

    # Term 1: Intra-set distances for X1
    term1 = (1 / m**2) * rbf_batch(X1, X1, sigma)

    # Term 2: Intra-set distances for X2
    term2 = (1 / n**2) * rbf_batch(X2, X2, sigma)

    # Term 3: Inter-set distances between X1 and X2
    term3 = (2 / (m * n)) * rbf_batch(X1, X2, sigma)

    d = term1 + term2 - term3
    return d

In [8]:
def pct_var(X, V):
    """
    Compute the % of variance retained by the projection in the reduced dataset
    """
    
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    
    return 100 * np.trace(V.T @ Sigma @ V) / np.trace(Sigma) 

In [9]:
def pct_accuracy(X_reduced_data, Y1_labels, kernel='rbf', C=1.0, gamma='scale', cv=5):
    """
    Compute the cross-validated accuracy (%Acc) for a kernel SVM.
    
    Parameters:
    - reduced_data: np.ndarray, the reduced data matrix (n_samples, n_features)
    - labels: np.ndarray, the target labels (n_samples,)
    - kernel: str, the kernel type to use for SVM ('linear', 'rbf', etc.)
    - C: float, regularization parameter for SVM
    - gamma: str or float, kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    - cv: int, the number of cross-validation folds

    Returns:
    - mean_accuracy: float, mean accuracy across folds
    - std_accuracy: float, standard deviation of accuracy across folds
    """
    svm = SVC(kernel=kernel, C=C, gamma=gamma)
    accuracy_scores = cross_val_score(svm, X_reduced_data, Y1_labels, cv=cv, scoring='accuracy')
    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)
    return mean_accuracy * 100, std_accuracy * 100

In [10]:
def pct_delta_demographic_parity(X_reduced_data, Y1_labels, Y2_groups, kernel='rbf', C=1.0, gamma='scale', cv=5):
    """
    Compute %∆DP (Demographic Parity) for a kernel SVM.

    Parameters:
    - X_reduced_data: np.ndarray, the reduced data matrix (n_samples, n_features)
    - Y2_labels: np.ndarray, the target labels (n_samples,)
    - groups: np.ndarray, the demographic group labels (n_samples,)
    - kernel: str, the kernel type to use for SVM ('linear', 'rbf', etc.)
    - C: float, regularization parameter for SVM
    - gamma: str or float, kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    - cv: int, the number of cross-validation folds

    Returns:
    - mean_dp: float, mean demographic parity (%∆DP) across folds
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    svm = SVC(kernel=kernel, C=C, gamma=gamma)

    dp_differences = []

    for train_index, test_index in skf.split(X_reduced_data, Y1_labels):
        X_train, X_test = X_reduced_data[train_index], X_reduced_data[test_index]
        y_train, y_test = Y1_labels[train_index], Y1_labels[test_index]
        g_test = Y2_groups[test_index]

        svm.fit(X_train, y_train)
        y_pred = svm.predict(X_test)

        # Compute positive prediction rates per group
        positive_rates = {}
        for group in np.unique(g_test):
            group_indices = np.where(g_test == group)[0]
            positive_rate = np.mean(y_pred[group_indices] == 1)
            positive_rates[group] = positive_rate

        # Compute absolute differences between groups
        group_rates = list(positive_rates.values())
        dp_difference = abs(group_rates[0] - group_rates[1])  # Adjust for two-group assumption
        dp_differences.append(dp_difference)

    mean_dp = np.mean(dp_differences) * 100  # Convert to percentage
    sd_dp = np.std(dp_differences) * 100 # Convert to percentage
    return mean_dp, sd_dp

# Load CelebA Data

In [58]:
Y_subset = np.load("Y_subset.npy")
Z_subset = np.load("Z_subset.npy")

In [59]:
X_subset = np.load("X_subset.npy")
#X_subset2 = np.load("X_subset2.npy")
#X_subset3 = np.load("X_subset3.npy")

In [72]:
X_normalized = (X_subset - 127.5) / 127.5  # Scale to [-1,1]

In [60]:
from sklearn.preprocessing import StandardScaler

# Initialize a StandardScaler for each subset
#scaler1 = StandardScaler()
#scaler2 = StandardScaler()
#scaler3 = StandardScaler()

# Fit and transform each subset separately
#X_subset1_scaled = scaler1.fit_transform(X_subset1)
#X_subset2_scaled = scaler2.fit_transform(X_subset2)
#X_subset3_scaled = scaler3.fit_transform(X_subset3)

In [ ]:
import time

start_time = time.time()

V = method1(X_normalized, Y_subset, Z_subset, 500, 0.01, 10)

#V1 = method1(X_subset1_scaled, Y_subset, Z_subset, 20, 0.5, 10)
#V2 = method1(X_subset2_scaled, Y_subset, Z_subset, 20, 0.5, 10)
#V3 = method1(X_subset3_scaled, Y_subset, Z_subset, 20, 0.5, 10)

elapsed_time = time.time() - start_time
print(f"method1 execution time: {elapsed_time:.2f} seconds")

# Time the saving process
save_start_time = time.time()

In [ ]:
np.save("V_prenormalized_celebAd=500_.01_10.npy", V)
#np.save("V2_celebAd=20_0.5_10.npy", V2)
#np.save("V3_celebAd=20_0.5_10.npy", V3)
save_elapsed_time = time.time() - save_start_time
print(f"Saving time: {save_elapsed_time:.2f} seconds")

In [32]:
#V1V1T = V1 @ V1.T

In [75]:
VVT = V @ V.T

In [17]:
#V2V2T = V2 @ V2.T

In [18]:
#V3V3T = V3 @ V3.T

In [76]:
X_mean = X_subset.mean(axis=0)  # Compute the mean of the original data
X_centered = X_normalized - X_mean  # Center the data

X_reduced = (X_centered @ VVT) + X_mean  # Apply projection and add back the mean

In [20]:
#X_subset1_reduced = X_subset1 @ V1V1T

In [21]:
#X_subset2_reduced = X_subset2 @ V2V2T

In [22]:
#X_subset3_reduced = X_subset3 @ V3V3T

In [ ]:
np.min(X_subset)

In [ ]:
np.max(X_subset)

In [ ]:
np.min(X_reduced)

In [ ]:
np.max(X_reduced)

In [77]:
low, high = np.percentile(X_reduced, [1, 99])  # Get robust min/max
X_reduced = np.clip(X_reduced, low, high)  # Clip extreme outliers
X_reduced = (X_reduced - low) / (high - low) * 255  # Rescale to 0-255

In [65]:
mean = X_reduced.mean()
std = X_reduced.std()
X_reduced = np.clip(X_reduced, mean - 2 * std, mean + 2 * std)  # Limit extreme values
X_reduced = (X_reduced - X_reduced.min()) / (X_reduced.max() - X_reduced.min()) * 255

In [28]:
# Min-max normalization to range 0-255
arr_min = X_reduced.min()
arr_max = X_reduced.max()
rescaled_arr = (X_reduced - arr_min) / (arr_max - arr_min) * 255

# Min-max normalization to range 0-255
#arr_min = X_subset2_reduced.min()
#arr_max = X_subset2_reduced.max()
#rescaled_arr2 = (X_subset2_reduced - arr_min) / (arr_max - arr_min) * 255

# Min-max normalization to range 0-255
#arr_min = X_subset3_reduced.min()
#arr_max = X_subset3_reduced.max()
#rescaled_arr3 = (X_subset3_reduced - arr_min) / (arr_max - arr_min) * 255

#X_combined = np.dstack([rescaled_arr1, rescaled_arr2, rescaled_arr3])
#X_combined = X_combined.reshape(20000, 29700)  # Flatten for consistency

In [78]:
np.save('X_grayscale_prenormalized_rescaled2_d=500_alpha=.01_lambda=10.npy', X_reduced)

In [ ]:
eigh(VVT)